## Causal Discovery in Serum miRNomes: Revealing Biomarker Networks for Early Cancer Prediction

In [1]:
import pandas as pd

dat = pd.read_csv('data.txt', sep = '\t')

### Outcome

BC: Breast cancer

BL: Bladder cancer

BT: Biliary tract cancer

CC: Colorectal cancer

EC: Esophageal squamous cell carcinoma

GC: Gastric cancer

HC: Hepatocellular cancer

LK: Lung cancer

OV: Ovarian cancer

PC: Pancreatic cancer

PR: Prostate cancer

SA: Sarcoma

In [2]:
import numpy as np
import re
from collections import Counter

y = ["".join(re.findall(r'[A-Za-z]+', each)) for each in dat.columns[1:]]

cancerlist = ['BC','BL','BT','CC','EC','GC','HC','LK','OV','PC','PR','SA']
y = np.array([each if each in cancerlist else 'Control' for each in y])

### Count occurrences
name_counts = Counter(y)
for name, count in name_counts.items():
    print(f"{name}: {count}")

BC: 706
BL: 399
BT: 402
CC: 1596
Control: 5908
EC: 566
GC: 1418
HC: 348
LK: 1699
OV: 428
PC: 851
PR: 1257
SA: 612


In [3]:
X = dat.transpose()
X = X.rename(columns = X.iloc[0]) 
X = X.drop(X.index[0])             
X = X.reset_index(drop = True)

X

,hsa-miR-28-3p,hsa-miR-27a-5p,hsa-miR-518b,hsa-miR-520b,hsa-miR-498,hsa-miR-512-3p,hsa-miR-491-5p,hsa-miR-490-3p,hsa-miR-452-5p,hsa-miR-451a,...,hsa-miR-3606-3p,hsa-miR-1292-3p,hsa-miR-6889-5p,hsa-miR-6888-3p,hsa-miR-6881-5p,hsa-miR-6880-3p,hsa-miR-6873-5p,hsa-miR-6872-3p,hsa-miR-6865-5p,hsa-miR-6864-3p
0,-1.177703,-1.177703,-1.177703,-1.177703,2.839246,-1.177703,-1.177703,-1.177703,-1.177703,-1.177703,...,-1.177703,-1.177703,6.80529,-1.177703,-1.177703,0.016052,-1.177703,-1.177703,-1.177703,-1.177703
1,-1.556806,-1.556806,-1.556806,-1.556806,4.716967,-1.556806,-1.556806,-1.556806,-1.556806,-1.556806,...,-1.556806,3.944541,6.959148,-1.556806,-1.556806,5.41015,-1.556806,5.877905,1.045616,-1.556806
2,-1.321168,-1.321168,-1.321168,-1.321168,4.327931,-1.321168,-1.321168,-1.321168,-1.321168,6.477432,...,-1.321168,4.881499,6.729782,-1.321168,-1.321168,6.252127,-1.321168,4.664817,3.973792,-1.321168
3,-2.176251,-2.176251,-2.176251,-2.176251,4.970866,-2.176251,-2.176251,-2.176251,-2.176251,6.48354,...,-2.176251,3.02212,7.05241,-2.176251,-2.176251,5.719362,-2.176251,4.472891,4.943061,-2.176251
4,-2.079343,-2.079343,-2.079343,-2.079343,1.726077,-2.079343,-2.079343,-2.079343,-2.079343,2.708338,...,-2.079343,-0.747342,6.497237,-2.079343,-2.079343,5.140446,-2.079343,-2.079343,-2.079343,-2.079343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16185,3.867679,3.501322,2.837191,0.665455,4.803604,1.130736,6.193152,0.41655,-0.498561,1.665464,...,-5.540565,4.7634,7.220702,-5.540565,3.089055,4.941072,-0.742389,4.942069,4.134535,-5.540565
16186,5.311947,4.073188,4.101201,2.118686,5.698477,2.344266,4.712554,0.785063,-4.417322,2.849492,...,-4.417322,5.446265,7.163131,-4.417322,2.154039,4.884964,-4.417322,4.718529,3.350626,-4.417322
16187,2.229432,2.674183,3.867016,3.424473,5.689167,3.830466,5.090698,1.019828,0.885724,6.267623,...,-4.324501,5.002374,7.283329,-4.324501,2.489638,5.280215,0.143643,4.764032,3.781309,-1.050319
16188,2.249489,-0.639963,0.423182,1.028599,5.844544,0.472641,5.284079,-0.795367,-4.01753,5.966505,...,-4.01753,4.99631,7.052476,-4.01753,1.27836,4.922518,-0.396838,4.909489,3.475894,-4.01753


#### Construct dataset

In [4]:
from sklearn.ensemble import RandomForestClassifier

def dataset(name = 'CC', n_features = 50):
    
    ### Select data
    index = np.isin(y, [name, 'Control'])
    _X, _y = X.iloc[index,], y[index]
    _y = 1 * (_y != 'Control')
    
    ### Fit RF to get the top 100 important features
    clf = RandomForestClassifier(
        n_estimators = 200,
        random_state = 42,
    )
    clf.fit(_X, _y)

    importances = clf.feature_importances_
    idx_sorted = np.argsort(importances)[::-1]
    top_idx = idx_sorted[:n_features]
    top_features = _X.columns[top_idx]
    _X_reduced = _X[top_features].copy()
    
    return _X_reduced, _y

CC_X, CC_y = dataset('CC')
GC_X, GC_y = dataset('GC')
LU_X, LU_y = dataset('LK')

print('CC', CC_X.shape, len(CC_y))
print('GC', GC_X.shape, len(GC_y))
print('LK', LU_X.shape, len(LU_y))

CC (7504, 50) 7504
GC (7326, 50) 7326
LK (7607, 50) 7607


### CC

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(CC_X, CC_y, test_size = 0.3, random_state = 42)

### Initialize models
models = {
    'Logistic Regression': make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter = 1000, random_state = 42)
    ),
    'Linear SVM': make_pipeline(
        StandardScaler(),
        SVC(kernel = 'linear', probability = True, random_state = 42)
    ),
    'Random Forest': RandomForestClassifier(n_estimators = 100, random_state = 42),
    'Gradient Boosting': GradientBoostingClassifier(random_state = 42)
}

results = []
feature_importances = {}

### Train, predict, and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    prec = precision_score(y_test, y_pred)
    rec  = recall_score(y_test, y_pred)
    f1   = f1_score(y_test, y_pred)
    
    results.append({'Model': name, 'Precision': prec, 'Recall': rec, 'F1 Score': f1})
    
    if isinstance(model, Pipeline):
        model = model.steps[-1][1]
        
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        importances = abs(model.coef_)[0]
    
    feat_imp_series = pd.Series(importances, index = CC_X.columns)
    top10 = feat_imp_series.sort_values(ascending = False).head(10)
    feature_importances[name] = top10

### Display the results table
results_df = pd.DataFrame(results)
print('\nModel Performance:')
print(results_df.to_string(index = False))

### Display top 10 features for each model
for model_name, top_feats in feature_importances.items():
    print(f'\nTop 10 Features — {model_name}:')
    print(top_feats.to_string(header=['Importance']))


Model Performance:
              Model  Precision   Recall  F1 Score
Logistic Regression   0.986301 0.982456  0.984375
         Linear SVM   0.988281 0.986355  0.987317
      Random Forest   0.971374 0.992203  0.981678
  Gradient Boosting   0.971264 0.988304  0.979710

Top 10 Features — Logistic Regression:
hsa-miR-1228-5p    2.176648
hsa-miR-3180       1.654565
hsa-miR-4730       1.574396
hsa-miR-614        1.369362
hsa-miR-6787-5p    1.359926
hsa-miR-6717-5p    1.356550
hsa-miR-6765-5p    1.334836
hsa-miR-211-3p     1.212059
hsa-miR-6802-5p    1.211165
hsa-miR-885-3p     1.130723

Top 10 Features — Linear SVM:
hsa-miR-1228-5p    1.645746
hsa-miR-4730       1.130794
hsa-miR-6802-5p    1.078796
hsa-miR-3180       1.028580
hsa-miR-6787-5p    0.974077
hsa-miR-6717-5p    0.929998
hsa-miR-614        0.796794
hsa-miR-6756-5p    0.782767
hsa-miR-1469       0.726079
hsa-miR-4783-3p    0.724582

Top 10 Features — Random Forest:
hsa-miR-4783-3p    0.150539
hsa-miR-1228-5p    0.146640
hsa-miR-4

In [6]:
from causallearn.search.ConstraintBased.PC import pc

### Dataset
df = pd.concat([CC_X, pd.Series(CC_y, name = 'Y', index = CC_X.index)], axis = 1)
df = df.astype(float)
data = df.values
var_names = df.columns.tolist()

### PC algorithm
cg = pc(data, alpha = 0.05, labels = var_names, max_k = 2)
print(cg.G)

  0%|          | 0/51 [00:00<?, ?it/s]

Graph Nodes:
X1;X2;X3;X4;X5;X6;X7;X8;X9;X10;X11;X12;X13;X14;X15;X16;X17;X18;X19;X20;X21;X22;X23;X24;X25;X26;X27;X28;X29;X30;X31;X32;X33;X34;X35;X36;X37;X38;X39;X40;X41;X42;X43;X44;X45;X46;X47;X48;X49;X50;X51

Graph Edges:
1. X2 --> X1
2. X1 --> X4
3. X7 --> X1
4. X1 --> X15
5. X35 --> X1
6. X51 --> X1
7. X2 --> X16
8. X19 --> X2
9. X20 --> X2
10. X22 --> X2
11. X2 --> X26
12. X31 --> X2
13. X34 --> X2
14. X51 --> X2
15. X3 --> X9
16. X16 --> X3
17. X3 --> X19
18. X3 --- X26
19. X27 --> X3
20. X32 --> X3
21. X3 --- X38
22. X8 --> X4
23. X10 --> X4
24. X22 --> X4
25. X23 --> X4
26. X35 --> X4
27. X51 --> X4
28. X8 --> X5
29. X5 --> X14
30. X5 --> X25
31. X5 --> X29
32. X35 --> X5
33. X6 --> X7
34. X11 --> X6
35. X12 --> X6
36. X6 --> X37
37. X39 --> X6
38. X40 --> X6
39. X44 --> X6
40. X13 --> X7
41. X39 --> X7
42. X42 --> X7
43. X45 --> X7
44. X8 --> X13
45. X25 --> X8
46. X28 --> X8
47. X29 --> X8
48. X43 --> X8
49. X26 --> X9
50. X29 --> X9
51. X32 --> X9
52. X9 --- X38
53. X49 --> X9

In [7]:
pd.DataFrame(var_names).to_csv('CC_names.csv')

In [8]:
from causallearn.search.FCMBased import lingam

model = lingam.DirectLiNGAM()
model.fit(data)
pd.DataFrame(model.adjacency_matrix_).to_csv('CC_lingam.csv')

In [9]:
from notears.linear import notears_linear

W_est = notears_linear(data, lambda1 = 0.0001, loss_type = 'l2')

pd.DataFrame(W_est).to_csv('CC_notears.csv')

### GC

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(GC_X, GC_y, test_size = 0.3, random_state = 42)

### Initialize models
models = {
    'Logistic Regression': make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter = 1000, random_state = 42)
    ),
    'Linear SVM': make_pipeline(
        StandardScaler(),
        SVC(kernel = 'linear', probability = True, random_state = 42)
    ),
    'Random Forest': RandomForestClassifier(n_estimators = 100, random_state = 42),
    'Gradient Boosting': GradientBoostingClassifier(random_state = 42)
}

results = []
feature_importances = {}

### Train, predict, and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    prec = precision_score(y_test, y_pred)
    rec  = recall_score(y_test, y_pred)
    f1   = f1_score(y_test, y_pred)
    
    results.append({'Model': name, 'Precision': prec, 'Recall': rec, 'F1 Score': f1})
    
    if isinstance(model, Pipeline):
        model = model.steps[-1][1]
        
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        importances = abs(model.coef_)[0]
    
    feat_imp_series = pd.Series(importances, index = CC_X.columns)
    top10 = feat_imp_series.sort_values(ascending = False).head(10)
    feature_importances[name] = top10

### Display the results table
results_df = pd.DataFrame(results)
print('\nModel Performance:')
print(results_df.to_string(index = False))

### Display top 10 features for each model
for model_name, top_feats in feature_importances.items():
    print(f'\nTop 10 Features — {model_name}:')
    print(top_feats.to_string(header=['Importance']))


Model Performance:
              Model  Precision   Recall  F1 Score
Logistic Regression   0.993197 0.993197  0.993197
         Linear SVM   0.986517 0.995465  0.990971
      Random Forest   0.993213 0.995465  0.994337
  Gradient Boosting   0.990991 0.997732  0.994350

Top 10 Features — Logistic Regression:
hsa-miR-4730       1.865394
hsa-miR-3184-5p    1.603938
hsa-miR-1228-5p    1.310323
hsa-miR-6781-5p    0.911518
hsa-miR-663a       0.903049
hsa-miR-320a       0.837229
hsa-miR-6088       0.827025
hsa-miR-575        0.813656
hsa-miR-17-3p      0.786541
hsa-miR-1307-3p    0.670886

Top 10 Features — Linear SVM:
hsa-miR-3184-5p    0.922888
hsa-miR-4730       0.773812
hsa-miR-1228-5p    0.539167
hsa-miR-6088       0.495571
hsa-miR-1203       0.422070
hsa-miR-6802-5p    0.399914
hsa-miR-6781-5p    0.388534
hsa-miR-663a       0.346766
hsa-miR-320a       0.346199
hsa-miR-17-3p      0.340894

Top 10 Features — Random Forest:
hsa-miR-1228-5p    0.160978
hsa-miR-3184-5p    0.131608
hsa-miR-6

In [11]:
from causallearn.search.ConstraintBased.PC import pc

### Dataset
df = pd.concat([GC_X, pd.Series(GC_y, name = 'Y', index = GC_X.index)], axis = 1)
df = df.astype(float)
data = df.values
var_names = df.columns.tolist()

### PC algorithm
cg = pc(data, alpha = 0.05, labels = var_names, max_k = 2)
print(cg.G)

  0%|          | 0/51 [00:00<?, ?it/s]

Graph Nodes:
X1;X2;X3;X4;X5;X6;X7;X8;X9;X10;X11;X12;X13;X14;X15;X16;X17;X18;X19;X20;X21;X22;X23;X24;X25;X26;X27;X28;X29;X30;X31;X32;X33;X34;X35;X36;X37;X38;X39;X40;X41;X42;X43;X44;X45;X46;X47;X48;X49;X50;X51

Graph Edges:
1. X11 --> X1
2. X13 --> X1
3. X14 --> X1
4. X1 --> X17
5. X30 --> X1
6. X31 --> X1
7. X1 --> X32
8. X42 --> X1
9. X44 --> X1
10. X1 --> X45
11. X5 --> X2
12. X2 --> X6
13. X17 --> X2
14. X38 --> X2
15. X51 --> X2
16. X13 --> X3
17. X14 --> X3
18. X20 --> X3
19. X21 --> X3
20. X3 --> X22
21. X3 --> X23
22. X25 --> X3
23. X49 --> X3
24. X51 --> X3
25. X5 --> X4
26. X18 --> X4
27. X33 --> X4
28. X44 --> X4
29. X47 --> X4
30. X51 --> X4
31. X7 --> X5
32. X36 --> X5
33. X42 --> X5
34. X46 --> X5
35. X51 --> X5
36. X21 --> X6
37. X36 --> X6
38. X37 --> X6
39. X51 --> X6
40. X15 --> X7
41. X17 --> X7
42. X26 --> X7
43. X32 --> X7
44. X33 --> X7
45. X7 --> X35
46. X11 --> X8
47. X8 --> X12
48. X13 --> X8
49. X18 --> X8
50. X8 --> X28
51. X38 --> X8
52. X42 --> X8
53. X8 --> 

In [12]:
pd.DataFrame(var_names).to_csv('GC_names.csv')

In [13]:
from causallearn.search.FCMBased import lingam

model = lingam.DirectLiNGAM()
model.fit(data)
pd.DataFrame(model.adjacency_matrix_).to_csv('GC_lingam.csv')

In [14]:
from notears.linear import notears_linear

W_est = notears_linear(data, lambda1 = 0.0001, loss_type = 'l2')

pd.DataFrame(W_est).to_csv('GC_notears.csv')

### LU

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(LU_X, LU_y, test_size = 0.3, random_state = 42)

### Initialize models
models = {
    'Logistic Regression': make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter = 1000, random_state = 42)
    ),
    'Linear SVM': make_pipeline(
        StandardScaler(),
        SVC(kernel = 'linear', probability = True, random_state = 42)
    ),
    'Random Forest': RandomForestClassifier(n_estimators = 100, random_state = 42),
    'Gradient Boosting': GradientBoostingClassifier(random_state = 42)
}

results = []
feature_importances = {}

### Train, predict, and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    prec = precision_score(y_test, y_pred)
    rec  = recall_score(y_test, y_pred)
    f1   = f1_score(y_test, y_pred)
    
    results.append({'Model': name, 'Precision': prec, 'Recall': rec, 'F1 Score': f1})
    
    if isinstance(model, Pipeline):
        model = model.steps[-1][1]
        
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        importances = abs(model.coef_)[0]
    
    feat_imp_series = pd.Series(importances, index = CC_X.columns)
    top10 = feat_imp_series.sort_values(ascending = False).head(10)
    feature_importances[name] = top10

### Display the results table
results_df = pd.DataFrame(results)
print('\nModel Performance:')
print(results_df.to_string(index = False))

### Display top 10 features for each model
for model_name, top_feats in feature_importances.items():
    print(f'\nTop 10 Features — {model_name}:')
    print(top_feats.to_string(header=['Importance']))


Model Performance:
              Model  Precision   Recall  F1 Score
Logistic Regression   0.994275 0.998084  0.996176
         Linear SVM   0.994264 0.996169  0.995215
      Random Forest   0.988506 0.988506  0.988506
  Gradient Boosting   0.984733 0.988506  0.986616

Top 10 Features — Logistic Regression:
hsa-miR-6786-5p    2.357817
hsa-miR-4730       1.811242
hsa-miR-1228-5p    1.479116
hsa-miR-1268a      1.274868
hsa-miR-6781-5p    1.091330
hsa-miR-6729-5p    0.910640
hsa-miR-663a       0.846899
hsa-miR-3184-5p    0.828001
hsa-miR-320a       0.784851
hsa-miR-1307-3p    0.754085

Top 10 Features — Linear SVM:
hsa-miR-6786-5p    1.442459
hsa-miR-4730       1.046154
hsa-miR-1268a      0.867385
hsa-miR-1228-5p    0.849464
hsa-miR-3184-5p    0.497034
hsa-miR-6781-5p    0.484598
hsa-miR-320a       0.462556
hsa-miR-614        0.456553
hsa-miR-3158-5p    0.448912
hsa-miR-6742-5p    0.434073

Top 10 Features — Random Forest:
hsa-miR-6786-5p    0.163025
hsa-miR-1228-5p    0.162683
hsa-miR-6

In [16]:
from causallearn.search.ConstraintBased.PC import pc

### Dataset
df = pd.concat([LU_X, pd.Series(LU_y, name = 'Y', index = LU_X.index)], axis = 1)
df = df.astype(float)
data = df.values
var_names = df.columns.tolist()

### PC algorithm
cg = pc(data, alpha = 0.05, labels = var_names, max_k = 2)
print(cg.G)

  0%|          | 0/51 [00:00<?, ?it/s]

Graph Nodes:
X1;X2;X3;X4;X5;X6;X7;X8;X9;X10;X11;X12;X13;X14;X15;X16;X17;X18;X19;X20;X21;X22;X23;X24;X25;X26;X27;X28;X29;X30;X31;X32;X33;X34;X35;X36;X37;X38;X39;X40;X41;X42;X43;X44;X45;X46;X47;X48;X49;X50;X51

Graph Edges:
1. X14 --> X1
2. X1 --> X17
3. X1 --> X25
4. X38 --> X1
5. X40 --> X1
6. X1 --> X42
7. X48 --> X1
8. X2 --> X5
9. X2 --> X7
10. X8 --> X2
11. X14 --> X2
12. X40 --> X2
13. X46 --> X2
14. X51 --> X2
15. X10 --> X3
16. X3 --> X17
17. X3 --- X19
18. X23 --> X3
19. X28 --> X3
20. X3 --> X32
21. X13 --> X4
22. X4 --> X21
23. X22 --> X4
24. X42 --> X4
25. X47 --> X4
26. X51 --> X4
27. X28 --> X5
28. X35 --> X5
29. X43 --> X5
30. X49 --> X5
31. X50 --> X5
32. X6 --> X7
33. X13 --> X6
34. X14 --> X6
35. X15 --> X6
36. X20 --> X6
37. X27 --> X6
38. X36 --> X6
39. X46 --> X6
40. X15 --> X7
41. X31 --> X7
42. X35 --> X7
43. X40 --> X7
44. X51 --> X7
45. X12 --> X8
46. X21 --> X8
47. X8 --> X25
48. X26 --> X8
49. X40 --> X8
50. X44 --> X8
51. X45 --> X8
52. X48 --> X8
53. X9 --> 

In [17]:
pd.DataFrame(var_names).to_csv('LU_names.csv')

In [18]:
from causallearn.search.FCMBased import lingam

model = lingam.DirectLiNGAM()
model.fit(data)
pd.DataFrame(model.adjacency_matrix_).to_csv('LU_lingam.csv')

In [19]:
from notears.linear import notears_linear

W_est = notears_linear(data, lambda1 = 0.0001, loss_type = 'l2')

pd.DataFrame(W_est).to_csv('LU_notears.csv')